In [22]:
import datetime
import requests
import csv 

In [23]:
buildingId = "e384f534f12c984588d8868e"
SERVER_HOST = '192.168.181.225'
SERVER_HOST = 'localhost'
SERVER_PORT = 3000
SERVER_URL = f"http://{SERVER_HOST}:{SERVER_PORT}"

In [24]:

def server_request(method, path,json=None):
    base = '/admin'
    uri = SERVER_URL + base + path 
    response = requests.request(method,uri,json=json)
    return response

def parse_date(date_str):
    return datetime.datetime.fromisoformat(date_str.replace('Z', '+00:00'))


In [25]:

routeName = ""
version = "v2"

In [26]:
def get_routes_titles():
    get_routes_titles = f"/processing/routes/{buildingId}/titles"
    response = server_request('GET', get_routes_titles)
    return response.json()


In [27]:
def get_all_points():
    get_all_points =  f"/processing/map/{buildingId}"
    response = server_request('GET', get_all_points)
    res = response.json()
    data= res['data']
    specific_date = datetime.datetime(2024, 6, 17, tzinfo=datetime.timezone.utc)
    filtered_data = [d for d in data if parse_date(d['createdAt']) > specific_date]
    return filtered_data

In [29]:
import json
raw_points_data_filename =  "points.json"
filtered_data = get_all_points()
json_data = json.dumps(filtered_data, indent=4) 
with open(raw_points_data_filename, 'w') as f:
    f.write(json_data)

In [18]:
rows = []
for f in filtered_data:    
    v = f['version']
    bid = f['buildingId']
    uplo = f['createdAt']
    for point in f['points']:
        x, y, floor = point['x'], point['y'], point['floor']
        if point['magnetometerData']:
            m = point['magnetometerData']
            md = m['data']
            mx = md['x']
            my=md['y']
            mz = md['z']
        else:
            m = None
            md = None
            mx = None
            my = None
            mz = None
        if point['magnetometerUncalibData']:
            mu = point['magnetometerUncalibData']
            mud = mu['data']
            y_bias = mud['y_bias']
            x_bias = mud['x_bias']
            z_uncalib = mud['z_uncalib']
            z_bias = mud['z_bias']
            y_uncalib = mud['y_uncalib']
            x_uncalib = mud['x_uncalib']
        else:
            mu = None
            mud = None
            y_bias = None
            x_bias = None
            z_uncalib = None
            z_bias = None
            y_uncalib = None
            x_uncalib = None
        
        rows.append([v,bid , x, y, floor, mx,my,mz,y_bias,x_bias,z_uncalib,z_bias,y_uncalib,x_uncalib,uplo])
len(rows)

177

In [19]:
csv_file = 'mag_data.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['version','buildingId','x', 'y', 'floor', "mx","my","mz","my_bias","mx_bias","mz_uncalib","mz_bias","my_uncalib","mx_uncalib",'uploaddAt']) 
    writer.writerows(rows)
print(f'Data successfully written to {csv_file}')

Data successfully written to mag_data.csv


In [50]:
rows = []
for f in filtered_data:    
    v = f['version']
    bid = f['buildingId']
    uplo = f['createdAt']
    for point in f['points']:
        # if point['test']:
        #     continue
        x, y, floor = point['x'], point['y'], point['floor']
        if point['gpsData']:
            gps = point['gpsData']
            co = gps['coords']
            extra = gps['extras']

            timestamp = gps['timestamp']            
            acc = co['accuracy']
            long = co['longitude']
            la = co['latitude']
            alt = co['altitude']
            he = co['heading']
            sp = co['speed']
            meanCn = extra['meanCn0']
            maxCn0 = extra['maxCn0']
            satellites = extra['satellites']
            rows.append([v,bid , x, y, floor, timestamp,acc,long,la,alt,he,sp,meanCn,maxCn0,satellites,uplo])
        else:
            rows.append([v, bid ,x,y,floor,None,None,None,None,None,None,None,None,None,None,uplo])
len(rows)


In [52]:
csv_file = 'gps_data.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['version','buildingId','x', 'y', 'floor', 'timestamp','accuracy','longitude','latitude','altitude','heading','speed','meanCn0','maxCn0','satellites','uploaddAt']) 
    writer.writerows(rows)
print(f'Data successfully written to {csv_file}')

Data successfully written to gps_data.csv


In [53]:
rows = []
for f in filtered_data:    
    v = f['version']
    bid = f['buildingId']
    uplo = f['createdAt']
    for point in f['points']:
        x, y, floor = point['x'], point['y'], point['floor']
        if point['wifiData']:
            for wifi in point['wifiData']:
                ssid = wifi['SSID']
                level = wifi['level']
                timestamp = wifi['timestamp']
                frequency = wifi['frequency']
                capabilities = wifi['capabilities']
                bssid = wifi['BSSID']
                
                rows.append([v,bid,x, y, floor, ssid,level,timestamp,frequency,bssid,capabilities,uplo])
len(rows)                

In [55]:
csv_file = 'wifi_data.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['version','buiildingId','x', 'y', 'floor', 'SSID', 'level','timestamp','frequency','BSSID','capabilities','uploadedAt'])  # Write header
    writer.writerows(rows)
print(f'Data successfully written to {csv_file}')

Data successfully written to wifi_data.csv


In [31]:
get_maps_route = f'/processing/map/{buildingId}'
get_map_route = f'/processing/map/{buildingId}/{version}'
create_map_route  = f'/processing/map/{buildingId}'
create_path_route = f'/processing/route/{buildingId}'
get_path_route = f'/processing/route/{buildingId}/{routeName}'
get_paths_route = f'/processing/routes/{buildingId}'

In [33]:

response = server_request('GET', get_paths_route)
res = response.json()

In [35]:
data = res['data']

In [36]:

raw_routes_data_filename =  "routes.json"

json_data = json.dumps(data, indent=4) 
with open(raw_routes_data_filename, 'w') as f:
    f.write(json_data)

In [29]:
response = server_request('POST', create_map_route,json= {
    'data' : {
        "buildingId":buildingId,
        "version":version,
        'points':[]
    }
})
response.status_code


201

In [33]:
response = server_request('GET', get_map_route)
response.json()


{'data': {'_id': '666b312a35d2c5b1c5c9c707',
  'buildingId': 'E384F534F12C984588D8868E',
  'version': 'v2',
  'points': [],
  '__v': 0,
  'createdAt': '2024-06-13T17:49:30.242Z',
  'updatedAt': '2024-06-13T17:49:30.242Z'}}

In [45]:
response = server_request('GET', f'/processing/map/{buildingId}/v3')
response.json()

{'data': {'_id': '666b336535d2c5b1c5c9c74d',
  'buildingId': 'e384f534f12c984588d8868e',
  'version': 'v3',
  'points': [],
  '__v': 0,
  'createdAt': '2024-06-13T17:59:01.842Z',
  'updatedAt': '2024-06-13T17:59:01.842Z'}}

In [62]:
response = server_request('GET', get_maps_route)
s = response.json()
data = s['data']
for d in data:
    print(d['version'], "points" , len(points))
    points = d['points']
    for point in points:
        print("(",point['x'], ",",point['y'], ")")

v5stairsoutsideleftfloorzero points 0
( 37 , 95 )
( 37 , 95 )
( 37 , 95 )
( 37 , 95 )
( 37 , 160 )
( 37 , 160 )
( 37 , 160 )
( 37 , 160 )
( 52 , 220 )
( 52 , 220 )
( 52 , 220 )
( 52 , 220 )
v1 points 12
( 730 , 496 )
v3 points 1
v4 points 0


v5stairsoutsideleftfloorzero points 0
( 37 , 95 )
( 37 , 95 )
( 37 , 95 )
( 37 , 95 )
( 37 , 160 )
( 37 , 160 )
( 37 , 160 )
( 37 , 160 )
( 52 , 220 )
( 52 , 220 )
( 52 , 220 )
( 52 , 220 )
v1 points 12
( 730 , 496 )
v3 points 1
